In [1]:
!pip install haversine

In [2]:
import pandas as pd
import numpy as np
import haversine as hs

from haversine import Unit


In [3]:
pubs_data = pd.read_csv("/content/data-wrangling-pubs-full-clean.csv", index_col=[0])
police_data = pd.read_csv(
    "/content/data-wrangling-police_clean.csv", index_col=[0]
)
underground_data = pd.read_csv("/content/raw-london-underground.csv")
crime_data = pd.read_csv("/content/raw_london_sexual_crimes.csv", index_col=[0])


In [4]:
pubs_data.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating
0,4bc1e42eabf49521c690c193,The Black Horse,92 Wood St,London,Barnet,EN5 4BW,Pub,51.653075,-0.206657,0.939551,1.0,8.1
1,4b995bccf964a5209f7535e3,Ye Olde Mitre Inne,58 High St,Hertfordshire,Barnet,EN5 5SJ,Pub,51.652979,-0.199367,0.982768,1.0,7.6
2,4bb4da3fa7059521b8cc1bce,The Arkley,Barnet Rd,Barnet,Barnet,EN5 3EP,Pub,51.652533,-0.219573,0.987375,1.0,7.7
3,4dc436e5ae608779d11bd561,Railway Tavern,3 East Barnet Rd,Barnet,Barnet,EN4 8RR,Pub,51.650059,-0.174692,0.950895,1.0,7.4
4,4d72b1e78e12b1f793863f05,The Kings Head,84 High St,Barnet,Barnet,EN5 5SN,Pub,51.653499,-0.200979,0.377777,1.0,6.6


In [5]:
underground_data.head()

,Unnamed: 0,name,latitude,longitude,line
0,0,Acton Town,51.502500,-0.278126,"District, Piccadilly"
1,1,Acton Central,51.508835,-0.263033,London Overground
2,2,Acton Central,51.508560,-0.262880,London Overground
3,3,Aldgate,51.513940,-0.075370,Metropolitan
4,4,Aldgate East,51.515140,-0.071780,"District, Hammersmith & City"


In [6]:
police_data.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude
0,4f75c889e4b0b24b0c1c4c98,Barnet Police Station,26-32 High St,Barnet,Barnet,EN5 5RU,Police Station,51.652283,-0.198046
1,886e605708d446fd6af7be02,Barnet Police Station,NaN,London,NaN,NaN,Police Station,51.654345,-0.201383
2,7687cbfe5d9743d0bcfd91b6,Metropolitan Police,111 High St,Barnet,NaN,EN5 5XY,Police Station,51.655584,-0.202649
3,dd8ab317b29242cec99126f6,Cat Hill,"Snt Base, 13 Cat Hill",Barnet,NaN,EN4 8HG,Police Station,51.643170,-0.162268
4,fa389098b2a249fb9381e3f3,Metropolitan Police Service,13 Cat Hill,Barnet,NaN,EN4 8HG,Police Station,51.643139,-0.161936


In [7]:
crime_data.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
1,3f8f7ac887eaf9211b0ae601304b26722a8d19bbb5ba1b...,2021-12,City of London Police,City of London Police,-0.109640,51.520624,On or near Leather Lane,E01000916,Camden 027B,Violence and sexual offences,Under investigation,NaN
11,3f8f7ac887eaf9211b0ae601304b26722a8d19bbb5ba1b...,2021-12,City of London Police,City of London Police,-0.097562,51.518864,On or near Parking Area,E01000001,City of London 001A,Violence and sexual offences,Unable to prosecute suspect,NaN
21,fcd78bb549a08c5f2cbfbb0c70b6ba8e5dd61fa8ebe1b9...,2021-12,City of London Police,City of London Police,-0.095026,51.518499,On or near Park/Open Space,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN
22,f4afa08625bdc8161e9dc0b14b39049581e5bc3a643e6f...,2021-12,City of London Police,City of London Police,-0.092628,51.517920,On or near St Alphage Garden,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN
23,419176d79450dc395d689eed75c05ec59b6e84b7d43159...,2021-12,City of London Police,City of London Police,-0.092723,51.517032,On or near Aldermanbury Square,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN


In [8]:
crime_data.rename(
    {
        "Crime ID": "crime_id",
        "Month": "month",
        "Reported by": "reported_by",
        "Falls within": "falls_within",
        "Longitude": "longitude",
        "Latitude": "latitude",
        "Location": "location",
        "Crime type": "crime_type",
    },
    axis=1,
    inplace=True,
)


In [9]:
def get_distance(latitude1, longitude1, df):
    loc1 = (latitude1, longitude1)
    distance_min = 1000000
    for latitude2, longitude2 in list(zip(df.latitude, df.longitude)):
        loc2 = (latitude2, longitude2)
        distance = hs.haversine(loc1, loc2, unit=Unit.METERS)
        if distance < distance_min:
            distance_min = distance
    return distance_min


In [10]:
pubs_data["distance_metro"] = pubs_data.apply(
    lambda row: get_distance(row["latitude"], row["longitude"], underground_data),
    axis=1,
)


In [11]:
pubs_data["distance_police"] = pubs_data.apply(
    lambda row: get_distance(row["latitude"], row["longitude"], police_data), axis=1
)


In [12]:
pubs_data.sort_values("distance_metro", ascending=False).head(10)


,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police
74,4b65a1dcf964a52032f72ae3,The Royal Oak,High St,Green Street West,Orpington,BR6 6BJ,Pub,51.351319,0.089519,0.904898,1.0,7.0,16421.666549,245.869158
78,4b9e9c0ff964a5208ff336e3,The Queens Head,73 High St,Orpington,Orpington,BR6 6BQ,Pub,51.352253,0.089814,0.988041,1.0,6.2,16343.800356,140.243101
81,4e0fa1768877936cb411719f,The Buff,Pinewood Dr,Orpington,NaN,BR6 9NL,Pub,51.360007,0.087786,0.935304,1.0,5.7,15539.261192,739.373431
79,4c67dac7b80abe9accd4d0e5,Change of Horses,87 High St,Orpington,Orpington,BR6 7BB,Beer Garden,51.359575,0.069915,0.935502,1.0,NaN,14944.900225,1551.184655
73,4e6886ebb3ad5d919752d2db,The Woodman,50 High St,Kent,Orpington,BR6 7BA,Bingo Center,51.359255,0.067621,0.938068,1.0,6.6,14900.304645,1682.199072
84,4bd2d1a8caff95210979d3f0,The Maxwell,Station Rd,Bromley,Orpington,BR6 0RZ,Pub,51.371829,0.090791,0.928753,1.0,NaN,14574.936701,900.151097
85,4c87abae821e9eb095cd8d89,The Harvest Moon,141-143 High St,Orpington,Orpington,BR6 0LQ,Pub,51.377816,0.101193,0.957069,1.0,5.0,14499.138570,86.023654
75,4b5b61f4f964a52088f928e3,Ye Olde Whyte Lyon,Farnborough Common,Kent,Orpington,BR6 8NE,Pub,51.364223,0.056685,0.909086,1.0,6.4,14060.927978,2603.929158
71,4b5b7847f964a520a20029e3,The British Queen,427 Crofton Rd,Orpington,Orpington,BR6 8NL,Pub,51.366437,0.054200,0.946402,1.0,7.3,13763.706988,2873.362006
69,4c8d2503509e370492a43955,Black Horse,Crofton Rd,Orpington,Orpington,BR6 8NW,Pub,51.368235,0.056846,0.954741,1.0,7.5,13668.240422,2826.749082


In [13]:
pubs_data.sort_values("distance_police", ascending=False).head(10)


,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police
77,4d04c89ea268548102bab9bd,The Fox Inn,Heathfield Rd,Keston,Keston,BR2 6BQ,Pub,51.361966,0.027390,0.938291,1.0,6.4,13509.864815,4450.039902
72,4b75bb96f964a5208d202ee3,The Greyhound,4 Commonside,Hayes,Keston,BR2 6BP,Pub,51.363645,0.028593,0.956841,1.0,7.4,13364.667697,4412.322506
76,4c0bf149340720a101e18893,The Chequers,177 Southborough Lane,Bromley,Bromley,BR2 8AP,Pub,51.390280,0.045439,0.975612,1.0,6.5,11135.478054,4109.081646
82,4c781d0e566db60cc587400e,The Two Doves,37 Oakley Rd,London,Bromley,BR2 8HD,Arts and Entertainment,51.372590,0.039601,0.959509,1.0,NaN,12715.578214,4094.024325
696,4b58b3d9f964a5207b6628e3,The Tichenham Inn,11 Swakeleys Rd,Ickenham,Uxbridge,UB10 8DF,Arts and Entertainment,51.564438,-0.444744,0.959818,1.0,5.4,362.630888,3032.167296
638,4b65efd0f964a52095092be3,The Moon & Sixpence,250 Uxbridge Rd,Hatch End,Pinner,HA5 4HS,Pub,51.608274,-0.373095,0.979877,1.0,6.1,1806.176667,2997.865839
891,4b000e9ef964a520953a22e3,The Crown,117 Burnt Ash Hill,London,Lee,SE12 0AJ,Pub,51.443711,0.017690,0.977309,1.0,6.3,5173.759942,2922.538201
707,4bd3338477b29c7479319082,The Plough Inn,Tentelow Lane,Norwood Green,Southall,UB2 4LG,Pub,51.494919,-0.365912,0.984589,1.0,6.6,1771.098095,2879.429542
71,4b5b7847f964a520a20029e3,The British Queen,427 Crofton Rd,Orpington,Orpington,BR6 8NL,Pub,51.366437,0.054200,0.946402,1.0,7.3,13763.706988,2873.362006
69,4c8d2503509e370492a43955,Black Horse,Crofton Rd,Orpington,Orpington,BR6 8NW,Pub,51.368235,0.056846,0.954741,1.0,7.5,13668.240422,2826.749082


In [14]:
def get_number_of_crimes(latitude1, longitude1, df):
    loc1 = (latitude1, longitude1)
    crimes = 0
    for latitude2, longitude2 in list(zip(df["latitude"], df["longitude"])):
        loc2 = (latitude2, longitude2)
        distance = hs.haversine(loc1, loc2, unit=Unit.METERS)
        if distance <= 500:
            crimes += 1
    return crimes


In [15]:
pubs_data["number_of_crimes"] = pubs_data.apply(
    lambda row: get_number_of_crimes(row["latitude"], row["longitude"], crime_data),
    axis=1,
)


In [16]:
pubs_data.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes
0,4bc1e42eabf49521c690c193,The Black Horse,92 Wood St,London,Barnet,EN5 4BW,Pub,51.653075,-0.206657,0.939551,1.0,8.1,904.722178,390.281423,30
1,4b995bccf964a5209f7535e3,Ye Olde Mitre Inne,58 High St,Hertfordshire,Barnet,EN5 5SJ,Pub,51.652979,-0.199367,0.982768,1.0,7.6,443.945965,119.561106,45
2,4bb4da3fa7059521b8cc1bce,The Arkley,Barnet Rd,Barnet,Barnet,EN5 3EP,Pub,51.652533,-0.219573,0.987375,1.0,7.7,1767.173867,1215.815963,2
3,4dc436e5ae608779d11bd561,Railway Tavern,3 East Barnet Rd,Barnet,Barnet,EN4 8RR,Pub,51.650059,-0.174692,0.950895,1.0,7.4,1343.248049,1149.621255,64
4,4d72b1e78e12b1f793863f05,The Kings Head,84 High St,Barnet,Barnet,EN5 5SN,Pub,51.653499,-0.200979,0.377777,1.0,6.6,568.292759,98.112834,40


In [ ]:
pubs_data.to_csv("data-wrangling-pubs_data_with_crimes.csv")